# 🔬 Fine-Tune microsoft/phi-2 mit LoRA auf Google Colab
Dieses Notebook lädt dein `train_data.csv` und trainiert `phi-2` mit PEFT/LoRA.

In [14]:
# ✅ GPU prüfen
!nvidia-smi

Sat May 17 14:08:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             32W /   70W |    3336MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
# 📦 Installation
!pip install -q peft transformers accelerate bitsandbytes datasets

In [16]:
# 📁 CSV-Datei hochladen
from google.colab import files
uploaded = files.upload()  # Lade train_data.csv hoch

Saving train_data.csv to train_data (5).csv


In [17]:
# 📊 Daten laden
import pandas as pd
df = pd.read_csv('train_data.csv')
df.head()

,prompt,response
0,The risk of heart failure is 10%. Alcohol doub...,"base risk: 10%, new absolute risk: number not ..."
1,The risk of infection is 2%. Smoking triples t...,"base risk: 2%, new absolute risk: number not d..."
2,The chance of cancer is 5%. Medication reduces...,"base risk: 5%, new absolute risk: number not d..."
3,Heart attack risk is 8%. Exercise cuts it in h...,"base risk: 8%, new absolute risk: number not d..."
4,The risk of stroke is 6%. Alcohol increases th...,"base risk: 6%, new absolute risk: number not d..."


In [18]:
# 🧠 Modell laden & vorbereiten
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

model_id = 'microsoft/phi-2'
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto'
)
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# 🔧 LoRA-Konfiguration
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, config)

In [20]:
# 📚 Dataset vorbereiten
from datasets import Dataset

dataset = Dataset.from_pandas(df[['prompt', 'response']])

def tokenize(example):
    model_input = tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    labels = tokenizer(
        example["response"],
        padding="max_length",
        truncation=True,
        max_length=32
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_ds = dataset.map(tokenize, batched=False)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [23]:
# 🚀 Training
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='phi2-lora-output',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    logging_steps=1,
    save_strategy='epoch',
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,10.026300
2,10.403100
3,9.849700
4,11.136500
5,10.594500
6,10.022400
7,10.327800
8,10.174200
9,9.459700
10,10.619800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=15, training_loss=10.195828056335449, metrics={'train_runtime': 11.2794, 'train_samples_per_second': 1.33, 'train_steps_per_second': 1.33, 'total_flos': 7635551846400.0, 'train_loss': 10.195828056335449, 'epoch': 3.0})

In [24]:
# 💾 Modell speichern
model.save_pretrained('phi2-lora-output')
tokenizer.save_pretrained('phi2-lora-output')

('phi2-lora-output/tokenizer_config.json',
 'phi2-lora-output/special_tokens_map.json',
 'phi2-lora-output/vocab.json',
 'phi2-lora-output/merges.txt',
 'phi2-lora-output/added_tokens.json',
 'phi2-lora-output/tokenizer.json')

In [ ]:
# 🧪 Modell testen
from transformers import pipeline
pipe = pipeline('text-generation', model='phi2-lora-output', tokenizer=tokenizer)
pipe('The risk of diabetes is 7%. Lack of sleep increases it by 30%.', max_new_tokens=30)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]